<a href="https://colab.research.google.com/github/rushilbhat/AIMO/blob/main/RM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install datasets transformers torch bitsandbytes peft
# !pip install datasets transformers torch bitsandbytes peft xformers trl accelerate

!pip install packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes datasets transformers
# !pip install --upgrade accelerate
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [2]:
import time
import re
import pandas as pd
import torch
import torch.nn as nn
from datasets import load_dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, AutoConfig, LlamaForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig, DataCollatorForSeq2Seq
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:
def extract_code(entry):
  code_match = re.search(r'<llm-code>(.*?)</llm-code>', entry['generated_solution'], re.DOTALL)
  if code_match:
      entry['generated_solution'] = code_match.group(1).strip()
  else:
    entry['generated_solution'] = ""
  return entry

def tokenize_and_label(entry):
    question = entry['question']
    generated_solution = entry['generated_solution']
    is_correct = entry['is_correct']

    input_text = f"{question} {generated_solution}"
    input_ids = tokenizer.encode(input_text)
    question_ids = tokenizer.encode(question)

    labels = [-100.0] * len(question_ids) + [1.0 if is_correct else 0.0] * (len(input_ids) - len(question_ids)) # don't need to worry about token that spans across the end of the question and the beginning of the generated solution
    return {"input_ids": input_ids, "labels": labels}

In [5]:
class CustomLinearLayer(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.special_token_id = config.bos_token_id
        self.verifier_head = nn.Linear(1, 1, bias=True, dtype=model.dtype)

    def forward(self, input_ids, attention_mask=None, labels=None, **kwargs):
      with torch.autocast(device_type='cuda', dtype=torch.float16):
        outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        special_token_logits = logits[:, :, self.special_token_id].unsqueeze(-1)
        verification_scores = self.verifier_head(special_token_logits).squeeze(-1)

        if labels is not None:
            mask = (labels != -100)
            labels = labels.float()
            loss_fct = nn.MSELoss()
            verification_loss = loss_fct(verification_scores[mask], labels[mask])
            outputs.loss = verification_loss

        return outputs

In [6]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = torch.float16 # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "deepseek-ai/deepseek-math-7b-rl", # Choose ANY! eg mistralai/Mistral-7B-Instruct-v0.2
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
deepseek-ai/deepseek-math-7b-rl does not have a padding token! Will use pad_token = <|PAD_TOKEN|>.
Unsloth 2024.5 patched 30 layers with 30 QKV layers, 30 O layers and 30 MLP layers.


In [7]:
from google.colab import drive
drive.mount('/content/drive')
verifier_dataset = load_from_disk('/content/drive/My Drive/verifier_dataset')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
model_name = "deepseek-ai/deepseek-math-7b-rl"

config = AutoConfig.from_pretrained(model_name)
# nf4_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
# model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=nf4_config)

# model = prepare_model_for_kbit_training(model)

# lora_config = LoraConfig(
#     task_type=TaskType.CAUSAL_LM,   # Task type
#     r=8,                           # Rank of the low-rank matrices
#     lora_alpha=1,                 # Alpha scaling parameter
#     target_modules=['q_proj', 'v_proj'], # Target modules for LoRA
#     lora_dropout=0.1               # Dropout for LoRA
# )
# model = get_peft_model(model, lora_config)



# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


# dataset = load_dataset('nvidia/OpenMathInstruct-1', split='train')
# math_dataset = dataset.filter(lambda entry: entry['dataset'] == 'math')
# pot_math_dataset = math_dataset.filter(lambda entry: entry['error_message']=='').map(extract_code)
# verifier_dataset = pot_math_dataset.select(range(100000)).map(tokenize_and_label, remove_columns=pot_math_dataset.column_names)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)

In [9]:
verifier = CustomLinearLayer(model)

In [10]:
training_args = TrainingArguments(
    output_dir="verifier_output",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=8,
    learning_rate=1e-5,
)

trainer = Trainer(
    model=verifier,
    args=training_args,
    train_dataset=verifier_dataset,
    data_collator = data_collator
)

In [11]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 8
\        /    Total batch size = 256 | Total steps = 390
 "-____-"     Number of trainable parameters = 37,478,402


Step,Training Loss


KeyboardInterrupt: 

In [10]:
verifier_dataset

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 100000
})

In [ ]:
from torch.utils.data import DataLoader

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=None)
train_dataloader = DataLoader(verifier_dataset, batch_size=4, collate_fn=data_collator)

In [ ]:
first_batch = next(iter(train_dataloader))

In [ ]:
first_batch['attention_mask'][0]

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
first_batch['input_ids'][0]

tensor([100000,    774,    245,  22069,  15043,   1317,     11,    856,    317,
           254,   1604,    280,   8650,    279,    254,   5013,    473,    254,
          6947,    276,    254,   1420,  65026,     16,     20,     11,    207,
            23,   1026,     30,    473,   4300,   4027,   1666,  83130,    185,
           185,     87,     16,     11,    320,     16,     11,   1376,     17,
            11,    320,     17,    403,    207,     15,     11,    207,     15,
            11,    570,     16,     20,     11,    207,     23,    185,    185,
         20457,    403,  83130,   6034,     87,     17,    570,   1376,     16,
             8,    746,     17,    919,    334,     88,     17,    570,    320,
            16,      8,    746,     17,      8,    185,  20457, 100001, 100001,
        100001, 100001, 100001, 100001, 100001, 100001, 100001, 100001, 100001,
        100001, 100001, 100001, 100001, 100001, 100001, 100001, 100001, 100001,
        100001, 100001, 100001, 100001, 

In [ ]:
first_batch['labels'][0]

tensor([-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100,
        -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100])

In [ ]:
pot_math_dataset_sub1 = pot_math_dataset.select(range(100000)).map(tokenize)
pot_math_dataset_sub2 = pot_math_dataset.select(range(100000,200000)).map(tokenize)
pot_math_dataset_sub3 = pot_math_dataset.select(range(200000,300000)).map(tokenize)

df1 = pd.DataFrame(pot_math_dataset_sub1)
df2 = pd.DataFrame(pot_math_dataset_sub2)
df3 = pd.DataFrame(pot_math_dataset_sub3)

print(df1['tokens'].sum()/100000)
print(df2['tokens'].sum()/100000)
print(df3['tokens'].sum()/100000)

1463566 * 79

In [ ]:
torch.cuda.empty_cache()